This was ran in the kaggle environment. It's a kaggle notebook


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
from keras import layers, regularizers, optimizers
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Add, Flatten, Input, Activation, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Lambda, concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import keras.backend as K
import re
import math
import matplotlib.pyplot as plt
from tqdm import tqdm

# **Preprocessing Data**

In [ ]:
train = pd.read_csv('/kaggle/input/uw-cs480-fall20/train.csv')
test= pd.read_csv('/kaggle/input/uw-cs480-fall20/test.csv')
labels = train.category.factorize()
y = to_categorical(labels[0])
X_train, X_validation, y_train, y_validation = train_test_split(train, y, random_state=10, test_size=0.10)

In [ ]:
X_train, X_validation = X_train.reset_index(), X_validation.reset_index()


In [ ]:
train_image = []
validation_image=[]
test_image = []
for i in tqdm(range(X_train.shape[0])):
    img = image.load_img('/kaggle/input/uw-cs480-fall20/suffled-images/shuffled-images/'+X_train['id'][i].astype('str')+'.jpg', target_size=(32,32,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
for i in tqdm(range(X_validation.shape[0])):
    img = image.load_img('/kaggle/input/uw-cs480-fall20/suffled-images/shuffled-images/'+X_validation['id'][i].astype('str')+'.jpg', target_size=(32,32,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    validation_image.append(img)
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/kaggle/input/uw-cs480-fall20/suffled-images/shuffled-images/'+test['id'][i].astype('str')+'.jpg', target_size=(32,32,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)

X_test_image= np.array(test_image)
X_train_image = np.array(train_image)
X_validation_image = np.array(validation_image)

100%|██████████| 21628/21628 [02:18<00:00, 155.74it/s]


# Using both Text and Image

### Text Processing

In [ ]:
import re
def clean_text(data):
    n = len(data)
    for i in range(n):
        data[i]=data[i].lower()
        data[i]=re.sub('[^A-Za-z" "]+', '', data[i])
        data[i]=data[i].rstrip()
    return data

In [ ]:
# Get clean text data
X_train_text= X_train["noisyTextDescription"]
X_train_text = clean_text(X_train_text)
X_validation_text= X_validation["noisyTextDescription"]
X_validation_text = clean_text(X_validation_text)
X_test_text= test["noisyTextDescription"]
X_test_text = clean_text(X_test_text)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# TOKENIZING Text
# The maximum number of words to be used.
MAX_NB_WORDS = 5000
# Max number of words in each name.
MAX_SEQUENCE_LENGTH = 10
# Generating Word tokens
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X_train_text)
word_index = tokenizer.word_index

In [ ]:
X_train_text = tokenizer.texts_to_sequences(X_train_text) # Here it takes each word from names and replaces it with it's corresponding number from our word_index
X_train_text = pad_sequences(X_train_text, maxlen=MAX_SEQUENCE_LENGTH) # it transforms list of sequences into numpy aray of shape (num_sequence,num_timestep) 
X_validation_text = tokenizer.texts_to_sequences(X_validation_text) # Here it takes each word from names and replaces it with it's corresponding number from our word_index
X_validation_text = pad_sequences(X_validation_text, maxlen=MAX_SEQUENCE_LENGTH) 
X_test_text = tokenizer.texts_to_sequences(X_test_text) # Here it takes each word from names and replaces it with it's corresponding number from our word_index
X_test_text = pad_sequences(X_test_text, maxlen=MAX_SEQUENCE_LENGTH) 


In [ ]:
## Creating textmodel 
EMBEDDING_DIM = 30
textmodel=Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
textmodel.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM,input_length=X_train_text.shape[1]))
textmodel.add(Dropout(0.2))
# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
textmodel.add(Conv1D(256,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
textmodel.add(GlobalMaxPooling1D())

# We add a Fully Connected hidden layer:
textmodel.add(Dense(512))
textmodel.add(Dropout(0.2))
textmodel.add(Activation('relu'))

### Image Processing

In [ ]:
activation="relu"
size=3
model_image = Sequential()
model_image.add(Conv2D(32, (size, size), padding='same',input_shape=(32,32,3)))
model_image.add(Activation(activation))
model_image.add(Conv2D(32, (size, size), padding='same'))
model_image.add(Activation(activation))
model_image.add(MaxPooling2D(pool_size=(2, 2)))
model_image.add(Dropout(0.25))
model_image.add(Conv2D(64, (size, size), padding='same'))
model_image.add(Activation(activation))
model_image.add(Conv2D(64, (size, size), padding='same'))
model_image.add(Activation(activation))
model_image.add(MaxPooling2D(pool_size=(2, 2)))
model_image.add(Dropout(0.25))
model_image.add(Flatten())

### Final 

In [ ]:
merged=concatenate([model_image.output,textmodel.output]) ## Merging both the models
out=Dense(27)(merged) ## Adding a Dense layer to merged layers
out=Activation('softmax')(out) ## Adding activation softmax for classification
final_model=Model([model_image.input,textmodel.input],out) ## Final model which we will use for classification

In [ ]:
final_model.compile(loss='categorical_crossentropy',optimizer="Adam",metrics=['accuracy'])

In [ ]:
final_model.fit([X_train_image,X_train_text],y_train,epochs=30, validation_data=([X_validation_image, X_validation_text], y_validation))

Epoch 1/30
609/609 [==============================] - 74s 122ms/step - loss: 0.6686 - accuracy: 0.8191 - val_loss: 0.2458 - val_accuracy: 0.9334
Epoch 2/30
609/609 [==============================] - 73s 119ms/step - loss: 0.1806 - accuracy: 0.9511 - val_loss: 0.1849 - val_accuracy: 0.9538
Epoch 3/30
609/609 [==============================] - 73s 119ms/step - loss: 0.0905 - accuracy: 0.9765 - val_loss: 0.1903 - val_accuracy: 0.9505
Epoch 4/30
609/609 [==============================] - 74s 121ms/step - loss: 0.0578 - accuracy: 0.9842 - val_loss: 0.2012 - val_accuracy: 0.9519
Epoch 5/30
609/609 [==============================] - 74s 122ms/step - loss: 0.0419 - accuracy: 0.9888 - val_loss: 0.2405 - val_accuracy: 0.9478
Epoch 6/30
609/609 [==============================] - 74s 122ms/step - loss: 0.0306 - accuracy: 0.9903 - val_loss: 0.2381 - val_accuracy: 0.9487
Epoch 7/30
609/609 [==============================] - 73s 120ms/step - loss: 0.0260 - accuracy: 0.9929 - val_loss: 0.2556 - val_ac

### Predictions

In [ ]:
prediction3 = final_model.predict([X_test_image, X_test_text])
prediction3 = np.argmax(prediction3,axis=1)
predictions3 = labels[1][prediction3]
submission3 = pd.DataFrame({'id':test['id'],'category':predictions3})
submission3.to_csv('20603929_li.csv', header=True, index=False)